<a href="https://colab.research.google.com/github/AfrinDange/attendance_analysis/blob/main/scripts/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Git Repository

In [ ]:
!git clone https://github.com/AfrinDange/attendance_analysis.git

fatal: destination path 'attendance_analysis' already exists and is not an empty directory.


# Importing Libraries

In [148]:
import os
import pandas as pd
import re
from datetime import datetime
import csv
import xlrd
import json


# Roll Numbers

In [174]:
roll_no = pd.read_excel("attendance_analysis/data/roll_no.xlsx")
display(roll_no)
roll_no = roll_no.set_index('user_name').T.to_dict('roll_no')[0]

,roll_no,user_name
0,101710,CHRISTOPHER CARDOZA
1,101775,joel.thomas
2,101801,ALWIN JACOB
3,101802,B SAI SREE
4,101803,B SHRIRAM ATHREYA
...,...,...
67,101867,rahul.khambe
68,101868,vedesh.nadar
69,101869,shawn.periera
70,101870,gargi raje


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


# Sample Data

In [150]:
sample = pd.read_csv("attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3).csv", encoding = "UTF-16", skiprows=6, sep='\t')
display(sample)

,Full Name,Join Time,Leave Time,Duration,userPrincipalName,Role
0,Smita Dange,"8/12/2021, 4:32:58 PM","8/12/2021, 5:02:52 PM",29m 54s,smita.dange@fcrit.ac.in,Organizer
1,Smita Dange,"8/12/2021, 5:05:31 PM","8/12/2021, 5:32:29 PM",26m 57s,smita.dange@fcrit.ac.in,Organizer
2,rahul.khambe,"8/12/2021, 4:33:15 PM","8/12/2021, 5:31:33 PM",58m 17s,rahul.khambe@FCRIT.onmicrosoft.com,Presenter
3,PULHANI RACHIT,"8/12/2021, 4:33:35 PM","8/12/2021, 5:32:34 PM",58m 58s,pulhani.rachit@FCRIT.onmicrosoft.com,Presenter
4,BHAGYASHREE LAKSHMINARAYANAN,"8/12/2021, 4:33:36 PM","8/12/2021, 5:37:34 PM",1h 3m,bhagyashree.lakshmi@FCRIT.onmicrosoft.com,Presenter
...,...,...,...,...,...,...
69,THAKUR YUKTA,"8/12/2021, 4:43:26 PM","8/12/2021, 5:33:05 PM",49m 38s,thakur.yukta@FCRIT.onmicrosoft.com,Presenter
70,vedesh.nadar,"8/12/2021, 4:53:20 PM","8/12/2021, 5:44:01 PM",50m 41s,vedesh.nadar@FCRIT.onmicrosoft.com,Presenter
71,CHAVAN SHREYA,"8/12/2021, 4:56:25 PM","8/12/2021, 5:41:04 PM",44m 38s,chavan.shreya@FCRIT.onmicrosoft.com,Presenter
72,GUPTA ISHIKA,"8/12/2021, 5:13:32 PM","8/12/2021, 5:35:15 PM",21m 43s,gupta.ishika@FCRIT.onmicrosoft.com,Presenter


In [151]:
sample = pd.read_excel("attendance_analysis/data/MCC/MCC L23 26-08-2021.xlsx", skiprows=7)
display(sample)

,Full Name,Join Time,Leave Time,Duration,userPrincipalName,Role
0,Rakhi Kalantri,"8/26/2021, 10:11:23 AM","8/26/2021, 11:17:03 AM",1h 5m,rakhi.kalantri@fcrit.ac.in,Organizer
1,DESHMUKH PARTH,"8/26/2021, 10:15:35 AM","8/26/2021, 11:17:03 AM",1h 1m,deshmukh.parth@comp.fcrit.ac.in,Attendee
2,SERENA RAJU,"8/26/2021, 10:15:51 AM","8/26/2021, 11:17:03 AM",1h 1m,serena.raju@comp.fcrit.ac.in,Attendee
3,GEOFERRY MICHAEL,"8/26/2021, 10:16:41 AM","8/26/2021, 11:17:03 AM",1h,geoferry.michael@comp.fcrit.ac.in,Attendee
4,HUDIYA YASH,"8/26/2021, 10:16:48 AM","8/26/2021, 11:17:03 AM",1h,hudiya.yash@comp.fcrit.ac.in,Attendee
...,...,...,...,...,...,...
68,pramod more,"8/26/2021, 10:55:45 AM","8/26/2021, 10:55:56 AM",10s,pramod.more@fcrit.ac.in,Attendee
69,joel.thomas,"8/26/2021, 10:36:09 AM","8/26/2021, 11:17:03 AM",40m 54s,joel@FCRIT.onmicrosoft.com,Attendee
70,GUPTA CHIRANYA,"8/26/2021, 10:36:41 AM","8/26/2021, 11:17:03 AM",40m 22s,gupta.chiranya@comp.fcrit.ac.in,Attendee
71,GEORGE SIBIN,"8/26/2021, 10:39:05 AM","8/26/2021, 11:17:03 AM",37m 58s,george.sibin@comp.fcrit.ac.in,Attendee


# Convert Duration to Seconds

In [152]:
def time_parser(time):
    num_split = re.split("h|m|s", time)
    h_ind = time.find("h")
    m_ind = time.find("m")
    s_ind = time.find("s")

    secs = 0

    if h_ind != -1: # xh ... 
       secs += int(num_split[0])*60*60
    if m_ind != -1:
        if h_ind == -1: # xm ...
            secs += int(num_split[0])*60
        else: # xh ym ...
            secs += int(num_split[1])*60
    if s_ind != -1:
        if h_ind == -1: 
            if m_ind == -1: # xs
                secs += int(num_split[0])
            else: # xm ys
                secs += int(num_split[1])
        else: 
            if m_ind == -1: # xh ys
                secs += int(num_split[1])
            else: # xh ym zs
                secs += int(num_split[2])
    return secs

# Test Script

In [194]:
subject = "BDA"
dirPath = "attendance_analysis/data/"
all_data = []
for file in os.listdir(dirPath+subject):
    lecture_data = {}
    lecture_data['subject'] = subject
    if ".csv" in file:
        with open(dirPath+subject+"/"+file,'rt', encoding='utf16')as f:
            data = csv.reader(f, delimiter="\t")
            i = 0
            for row in data:
                if len(row) == 0:
                    skipped = i
                    break
                if row[0] == 'Total Number of Participants':
                    lecture_data['participants'] = int(row[1])
                if row[0] == 'Meeting Title':
                    lecture_data['meeting_title'] = row[1]
                if row[0] == 'Meeting Start Time':
                    lecture_data['meeting_start_time'] = row[1]
                if row[0] == 'Meeting End Time':
                    lecture_data['meeting_end_time'] = row[1]
                i = i + 1
        print(dirPath+subject+"/"+file)      
        lecture = pd.read_csv(dirPath+subject+"/"+file, encoding = "UTF-16", skiprows=skipped, sep="\t")
    else:
        wb = xlrd.open_workbook(dirPath+subject+"/"+file)
        sheet = wb.sheet_by_index(0)
        skipped = 0
        for i in range(0,sheet.nrows):
            row = sheet.row_values(i) 
            if row[0] == 'Total Number of Participants':
                lecture_data['participants'] = int(row[1])
            if row[0] == 'Meeting Title':
                lecture_data['meeting_title'] = row[1]
            if row[0] == 'Meeting Start Time':
                lecture_data['meeting_start_time'] = row[1]
            if row[0] == 'Meeting End Time':
                lecture_data['meeting_end_time'] = row[1]
            if row[0] == '':
                skipped = i+1
                break
        print(dirPath+subject+"/"+file)
        lecture = pd.read_excel(dirPath+subject+"/"+file, skiprows=skipped)
    attendees = {}
    for index,row in lecture.iterrows():
        if row['Full Name'] == 'pramod more':
            lecture_data['attendance_form_time'] = row['Join Time']
            break
        if row['Full Name'] not in attendees.keys():
            if row['Role'] == 'Presenter' or row['Role'] == 'Attendee':
                if row['Full Name'] in roll_no.keys():
                    attendee = {}
                    attendee['name'] = row['Full Name']
                    attendee['roll_no'] = roll_no[row['Full Name']]
                    attendee['join_time'] = row['Join Time']
                    attendee['leave_time'] = row['Leave Time']
                    attendee['duration'] = time_parser(row['Duration'])
                    try:
                        attendee['email'] = row['Email']
                    except KeyError:
                        try: 
                            attendee['email'] = row['Email (userPrincipalName)']
                        except KeyError:
                            attendee['email'] = row['userPrincipalName']
                    if attendee['email'] == '':
                        attendee['email'] = "-"
                    attendees[row['Full Name']] = attendee
            else:
                if 'organizer' not in lecture_data.keys():
                    organizer = {}
                    organizer['name'] = row['Full Name']
                    organizer['join_time'] = row['Join Time']
                    organizer['leave_time'] = row['Leave Time']
                    organizer['duration'] = time_parser(row['Duration'])
                    lecture_data['organizer'] = organizer
                else:
                    organizer = lecture_data['organizer']
                    prev_join_time = datetime.strptime(organizer['join_time'], '%m/%d/%Y, %I:%M:%S %p')
                    curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
                    if prev_join_time < curr_join_time: 
                        organizer['leave_time'] = row['Leave Time']
                    else:
                        organizer['join_time'] = row['Join Time']
                    organizer['duration'] += time_parser(row['Duration'])
                    lecture_data['organizer'] = organizer
                    
        else: # multiple join-leave
            attendee = attendees[row['Full Name']]
            prev_join_time = datetime.strptime(attendee['join_time'], '%m/%d/%Y, %I:%M:%S %p')
            curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
            if prev_join_time < curr_join_time: 
                attendee['leave_time'] = row['Leave Time']
            else:
                attendee['join_time'] = row['Join Time']
            attendee['duration'] += time_parser(row['Duration'])
    lecture_data['attendees'] = list(attendees.values())
    display(pd.DataFrame.from_dict(attendees))
    all_data.append(lecture_data)

attendance_analysis/data/BDA/meetingAttendanceReport(General) (2).csv


,FERNANDO EDWIN,WAREKAR ZAID,GUPTA ISHIKA,PULHANI RACHIT,gargi raje,BANSAL SAHIL,DESHMUKH PARTH,JASON FELIX,HANAH SUSAN,B SHRIRAM ATHREYA,DSOUZA ARNOLD,RAMESH ANGELA,DANY POLY,JUSTIN MATHEW,JOSEPH BENJAMIN,PINTO ALDRIC,CHARIVUKALAYIL JITIN,VERMA VARUN,SHYANN JAMES,JENNIFER JAMES,CHRISTOPHER CARDOZA,B SAI SREE,SAWANT VISHADH,PATIL GAURI,MULLICK SUDIKSHA,KARAKATTIL DILROSE,GOEL ESHAN,BHOLR ADITI,GOLE SIDDHANT,KADU NITISH,SINGH JASBIR,GEORGE SIBIN,RAGHUVANSHI AYUSH,HANDA RHEA,THAKUR YUKTA,MELEDATH GARY,BATTINA BABYKUTTY,SNEHA MARY,MERIN ANN,SHERIN SHIBU,SERENA RAJU,PADAMADAN SHAWN,PANDITA SHIVALIKA,BHAGDIKAR SOUMITRA,RIYA MOL,GEOFERRY MICHAEL,JERIN JOHN,SHARMA AKSHAT,shawn.periera,NAIR AKHIL,RADLEY ROY,BHAGYASHREE LAKSHMINARAYANAN,KURIAN SANMITH,CHAVAN SHREYA,RAINA HARSHUL,rahul.khambe,DANGE AFRIN,simran kalaskar,FERNANDES ARYAN,GUPTA CHIRANYA,vedesh.nadar,KAKUSTE ANSHUTA,joel.thomas,MEHTA SHIVAM,ALWIN JACOB,HUDIYA YASH
name,FERNANDO EDWIN,WAREKAR ZAID,GUPTA ISHIKA,PULHANI RACHIT,gargi raje,BANSAL SAHIL,DESHMUKH PARTH,JASON FELIX,HANAH SUSAN,B SHRIRAM ATHREYA,DSOUZA ARNOLD,RAMESH ANGELA,DANY POLY,JUSTIN MATHEW,JOSEPH BENJAMIN,PINTO ALDRIC,CHARIVUKALAYIL JITIN,VERMA VARUN,SHYANN JAMES,JENNIFER JAMES,CHRISTOPHER CARDOZA,B SAI SREE,SAWANT VISHADH,PATIL GAURI,MULLICK SUDIKSHA,KARAKATTIL DILROSE,GOEL ESHAN,BHOLR ADITI,GOLE SIDDHANT,KADU NITISH,SINGH JASBIR,GEORGE SIBIN,RAGHUVANSHI AYUSH,HANDA RHEA,THAKUR YUKTA,MELEDATH GARY,BATTINA BABYKUTTY,SNEHA MARY,MERIN ANN,SHERIN SHIBU,SERENA RAJU,PADAMADAN SHAWN,PANDITA SHIVALIKA,BHAGDIKAR SOUMITRA,RIYA MOL,GEOFERRY MICHAEL,JERIN JOHN,SHARMA AKSHAT,shawn.periera,NAIR AKHIL,RADLEY ROY,BHAGYASHREE LAKSHMINARAYANAN,KURIAN SANMITH,CHAVAN SHREYA,RAINA HARSHUL,rahul.khambe,DANGE AFRIN,simran kalaskar,FERNANDES ARYAN,GUPTA CHIRANYA,vedesh.nadar,KAKUSTE ANSHUTA,joel.thomas,MEHTA SHIVAM,ALWIN JACOB,HUDIYA YASH
roll_no,101817,101865,101823,101848,101870,101804,101814,101828,101825,101803,101815,101852,101813,101831,101806,101847,101810,101864,101859,101829,101710,101802,101855,101846,101842,101834,101820,101809,101821,101832,101861,101819,101850,101826,101863,101839,101805,101862,101840,101858,101856,101844,101845,101807,101853,101818,101830,101857,101869,101843,101849,101808,101836,101811,101851,101867,101812,101871,101816,101822,101868,101833,101775,101838,101801,101827
join_time,"7/30/2021, 9:01:20 AM","7/30/2021, 9:02:05 AM","7/30/2021, 9:02:27 AM","7/30/2021, 9:02:48 AM","7/30/2021, 9:03:21 AM","7/30/2021, 9:03:24 AM","7/30/2021, 9:03:27 AM","7/30/2021, 9:03:38 AM","7/30/2021, 9:03:52 AM","7/30/2021, 9:04:00 AM","7/30/2021, 9:04:10 AM","7/30/2021, 9:04:16 AM","7/30/2021, 9:04:21 AM","7/30/2021, 9:04:32 AM","7/30/2021, 9:04:40 AM","7/30/2021, 9:04:40 AM","7/30/2021, 9:04:45 AM","7/30/2021, 9:04:51 AM","7/30/2021, 9:04:52 AM","7/30/2021, 9:05:05 AM","7/30/2021, 9:05:05 AM","7/30/2021, 9:05:08 AM","7/30/2021, 9:05:14 AM","7/30/2021, 9:05:18 AM","7/30/2021, 9:05:21 AM","7/30/2021, 9:05:22 AM","7/30/2021, 9:05:23 AM","7/30/2021, 9:05:24 AM","7/30/2021, 9:05:24 AM","7/30/2021, 9:05:32 AM","7/30/2021, 9:05:38 AM","7/30/2021, 9:05:39 AM","7/30/2021, 9:05:44 AM","7/30/2021, 9:05:47 AM","7/30/2021, 9:05:53 AM","7/30/2021, 9:06:08 AM","7/30/2021, 9:06:08 AM","7/30/2021, 9:06:09 AM","7/30/2021, 9:06:12 AM","7/30/2021, 9:06:12 AM","7/30/2021, 9:06:16 AM","7/30/2021, 9:06:34 AM","7/30/2021, 9:06:44 AM","7/30/2021, 9:06:46 AM","7/30/2021, 9:06:54 AM","7/30/2021, 9:07:12 AM","7/30/2021, 9:07:21 AM","7/30/2021, 9:08:24 AM","7/30/2021, 9:08:53 AM","7/30/2021, 9:08:57 AM","7/30/2021, 9:09:50 AM","7/30/2021, 9:10:35 AM","7/30/2021, 9:10:40 AM","7/30/2021, 9:12:18 AM","7/30/2021, 9:12:25 AM","7/30/2021, 9:12:52 AM","7/30/2021, 9:13:25 AM","7/30/2021, 9:14:42 AM","7/30/2021, 9:16:12 AM","7/30/2021, 9:16:20 AM","7/30/2021, 9:17:48 AM","7/30/2021, 9:20:26 AM","7/30/2021, 9:24:20 AM","7/30/2021, 9:25:37 AM","7/30/2021, 9:29:58 AM","7/30/2021, 9:31:59 AM"
leave_time,"7/30/2021, 11:15:40 AM","7/30/2021, 11:15:49 AM","7/30/202

attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3).csv


,rahul.khambe,PULHANI RACHIT,BHAGYASHREE LAKSHMINARAYANAN,B SHRIRAM ATHREYA,WAREKAR ZAID,simran kalaskar,SANKANNAWAR PRIYANKA,DANGE AFRIN,KOTARKAR ANUJ,SERENA RAJU,gargi raje,BHAGDIKAR SOUMITRA,SHYANN JAMES,JERIN JOHN,DESHMUKH PARTH,NAIR AKHIL,CHARIVUKALAYIL JITIN,MEHTA SHIVAM,KURIAN SANMITH,SAWANT VISHADH,MULLICK SUDIKSHA,FERNANDO EDWIN,PINTO ALDRIC,SHARMA AKSHAT,JENNIFER JAMES,SNEHA MARY,KURUTHUKULANGARA JULIE,RADLEY ROY,KARAKATTIL DILROSE,VERMA VARUN,HANDA RHEA,RAMESH ANGELA,CHRISTOPHER CARDOZA,RIYA MOL,HANAH SUSAN,PANDITA SHIVALIKA,PADAMADAN SHAWN,JUSTIN MATHEW,SHERIN SHIBU,SINGH JASBIR,RAINA HARSHUL,GOEL ESHAN,GEORGE SIBIN,DANY POLY,GOLE SIDDHANT,FERNANDES ARYAN,BHOLR ADITI,MERIN ANN,BANSAL SAHIL,JOSEPH BENJAMIN,RAGHUVANSHI AYUSH,GEOFERRY MICHAEL,shawn.periera,PATIL GAURI,GUPTA CHIRANYA,MELEDATH GARY,DSOUZA ARNOLD,B SAI SREE,ALWIN JACOB,MERWIN SAMUEL,GUPTA PRANAY,joel.thomas,SINGH ASHISH,KADU NITISH,BATTINA BABYKUTTY,THAKUR YUKTA,vedesh.nadar,CHAVAN SHREYA,GUPTA ISHIKA,JASON FELIX
name,rahul.khambe,PULHANI RACHIT,BHAGYASHREE LAKSHMINARAYANAN,B SHRIRAM ATHREYA,WAREKAR ZAID,simran kalaskar,SANKANNAWAR PRIYANKA,DANGE AFRIN,KOTARKAR ANUJ,SERENA RAJU,gargi raje,BHAGDIKAR SOUMITRA,SHYANN JAMES,JERIN JOHN,DESHMUKH PARTH,NAIR AKHIL,CHARIVUKALAYIL JITIN,MEHTA SHIVAM,KURIAN SANMITH,SAWANT VISHADH,MULLICK SUDIKSHA,FERNANDO EDWIN,PINTO ALDRIC,SHARMA AKSHAT,JENNIFER JAMES,SNEHA MARY,KURUTHUKULANGARA JULIE,RADLEY ROY,KARAKATTIL DILROSE,VERMA VARUN,HANDA RHEA,RAMESH ANGELA,CHRISTOPHER CARDOZA,RIYA MOL,HANAH SUSAN,PANDITA SHIVALIKA,PADAMADAN SHAWN,JUSTIN MATHEW,SHERIN SHIBU,SINGH JASBIR,RAINA HARSHUL,GOEL ESHAN,GEORGE SIBIN,DANY POLY,GOLE SIDDHANT,FERNANDES ARYAN,BHOLR ADITI,MERIN ANN,BANSAL SAHIL,JOSEPH BENJAMIN,RAGHUVANSHI AYUSH,GEOFERRY MICHAEL,shawn.periera,PATIL GAURI,GUPTA CHIRANYA,MELEDATH GARY,DSOUZA ARNOLD,B SAI SREE,ALWIN JACOB,MERWIN SAMUEL,GUPTA PRANAY,joel.thomas,SINGH ASHISH,KADU NITISH,BATTINA BABYKUTTY,THAKUR YUKTA,vedesh.nadar,CHAVAN SHREYA,GUPTA ISHIKA,JASON FELIX
roll_no,101867,101848,101808,101803,101865,101871,101854,101812,101835,101856,101870,101807,101859,101830,101814,101843,101810,101838,101836,101855,101842,101817,101847,101857,101829,101862,101837,101849,101834,101864,101826,101852,101710,101853,101825,101845,101844,101831,101858,101861,101851,101820,101819,101813,101821,101816,101809,101840,101804,101806,101850,101818,101869,101846,101822,101839,101815,101802,101801,101841,101824,101775,101860,101832,101805,101863,101868,101811,101823,101828
join_time,"8/12/2021, 4:33:15 PM","8/12/2021, 4:33:35 PM","8/12/2021, 4:33:36 PM","8/12/2021, 4:33:37 PM","8/12/2021, 4:33:40 PM","8/12/2021, 4:33:41 PM","8/12/2021, 4:33:42 PM","8/12/2021, 4:33:43 PM","8/12/2021, 4:33:44 PM","8/12/2021, 4:33:46 PM","8/12/2021, 4:33:48 PM","8/12/2021, 4:33:49 PM","8/12/2021, 4:33:49 PM","8/12/2021, 4:33:49 PM","8/12/2021, 4:33:50 PM","8/12/2021, 4:33:50 PM","8/12/2021, 4:33:50 PM","8/12/2021, 4:33:54 PM","8/12/2021, 4:33:57 PM","8/12/2021, 4:33:57 PM","8/12/2021, 4:33:59 PM","8/12/2021, 4:34:01 PM","8/12/2021, 4:34:04 PM","8/12/2021, 4:34:04 PM","8/12/2021, 4:34:05 PM","8/12/2021, 4:34:10 PM","8/12/2021, 4:34:10 PM","8/12/2021, 4:34:15 PM","8/12/2021, 4:34:16 PM","8/12/2021, 4:34:18 PM","8/12/2021, 4:34:21 PM","8/12/2021, 4:34:21 PM","8/12/2021, 4:34:21 PM","8/12/2021, 4:34:22 PM","8/12/2021, 4:34:24 PM","8/12/2021, 4:34:27 PM","8/12/2021, 4:34:28 PM","8/12/2021, 4:34:29 PM","8/12/2021, 4:34:33 PM","8/12/2021, 4:34:33 PM","8/12/2021, 4:34:34 PM","8/12/2021, 4:34:40 PM","8/12/2021, 4:34:42 PM","8/12/2021, 4:34:43 PM","8/12/2021, 4:34:44 PM","8/12/2021, 4:34:47 PM","8/12/2021, 4:34:50 PM","8/12/2021, 4:34:58 PM","8/12/2021, 4:34:59 PM","8/12/2021, 4:35:02 PM","8/12/2021, 4:35:07 PM","8/12/2021, 4:35:10 PM","8/12/2021, 4:35:17 PM","8/12/2021, 4:35:41 PM","8/12/2021, 4:35:57 PM","8/12/2021, 4:36:04 PM","8/12/2021, 4:36:07 PM","8/12/2021, 4:36:26 PM","8/12/2021, 4:36:33 PM","8/12/2021, 4:37:07 PM","8/12/2021, 4:37:42 PM","8/12/2021, 4:38:08 PM

attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1) (1).csv


...

,B SAI SREE,simran kalaskar,MULLICK SUDIKSHA,FERNANDO EDWIN,SERENA RAJU,JUSTIN MATHEW,THAKUR YUKTA,CHRISTOPHER CARDOZA,HANAH SUSAN,JERIN JOHN,DANY POLY,DANGE AFRIN,GEOFERRY MICHAEL,gargi raje,JOSEPH BENJAMIN,BANSAL SAHIL,PINTO ALDRIC,KURIAN SANMITH,SAWANT VISHADH,JASON FELIX,SHERIN SHIBU,CHARIVUKALAYIL JITIN,GUPTA ISHIKA,RAMESH ANGELA,SHYANN JAMES,B SHRIRAM ATHREYA,GUPTA CHIRANYA,KAKUSTE ANSHUTA,SINGH ASHISH,VERMA VARUN,DSOUZA ARNOLD,PANDITA SHIVALIKA,rahul.khambe,BHOLR ADITI,FERNANDES ARYAN,PATIL GAURI,KADU NITISH,WAREKAR ZAID,GOEL ESHAN,RAGHUVANSHI AYUSH,SINGH JASBIR,GOLE SIDDHANT,MELEDATH GARY,MERIN ANN,BATTINA BABYKUTTY,GEORGE SIBIN,PULHANI RACHIT,BHAGDIKAR SOUMITRA,NAIR AKHIL,RAINA HARSHUL,SNEHA MARY,JENNIFER JAMES,KARAKATTIL DILROSE,SHARMA AKSHAT,BHAGYASHREE LAKSHMINARAYANAN,RADLEY ROY,DESHMUKH PARTH,SANKANNAWAR PRIYANKA,PADAMADAN SHAWN,joel.thomas,HUDIYA YASH,MEHTA SHIVAM,GUPTA PRANAY,KURUTHUKULANGARA JULIE,MERWIN SAMUEL,CHAVAN SHREYA
name,B SAI SREE,simran kalaskar,MULLICK SUDIKSHA,FERNANDO EDWIN,SERENA RAJU,JUSTIN MATHEW,THAKUR YUKTA,CHRISTOPHER CARDOZA,HANAH SUSAN,JERIN JOHN,DANY POLY,DANGE AFRIN,GEOFERRY MICHAEL,gargi raje,JOSEPH BENJAMIN,BANSAL SAHIL,PINTO ALDRIC,KURIAN SANMITH,SAWANT VISHADH,JASON FELIX,SHERIN SHIBU,CHARIVUKALAYIL JITIN,GUPTA ISHIKA,RAMESH ANGELA,SHYANN JAMES,B SHRIRAM ATHREYA,GUPTA CHIRANYA,KAKUSTE ANSHUTA,SINGH ASHISH,VERMA VARUN,DSOUZA ARNOLD,PANDITA SHIVALIKA,rahul.khambe,BHOLR ADITI,FERNANDES ARYAN,PATIL GAURI,KADU NITISH,WAREKAR ZAID,GOEL ESHAN,RAGHUVANSHI AYUSH,SINGH JASBIR,GOLE SIDDHANT,MELEDATH GARY,MERIN ANN,BATTINA BABYKUTTY,GEORGE SIBIN,PULHANI RACHIT,BHAGDIKAR SOUMITRA,NAIR AKHIL,RAINA HARSHUL,SNEHA MARY,JENNIFER JAMES,KARAKATTIL DILROSE,SHARMA AKSHAT,BHAGYASHREE LAKSHMINARAYANAN,RADLEY ROY,DESHMUKH PARTH,SANKANNAWAR PRIYANKA,PADAMADAN SHAWN,joel.thomas,HUDIYA YASH,MEHTA SHIVAM,GUPTA PRANAY,KURUTHUKULANGARA JULIE,MERWIN SAMUEL,CHAVAN SHREYA
roll_no,101802,101871,101842,101817,101856,101831,101863,101710,101825,101830,101813,101812,101818,101870,101806,101804,101847,101836,101855,101828,101858,101810,101823,101852,101859,101803,101822,101833,101860,101864,101815,101845,101867,101809,101816,101846,101832,101865,101820,101850,101861,101821,101839,101840,101805,101819,101848,101807,101843,101851,101862,101829,101834,101857,101808,101849,101814,101854,101844,101775,101827,101838,101824,101837,101841,101811
join_time,"7/27/2021, 9:00:54 AM","7/27/2021, 9:01:33 AM","7/27/2021, 9:01:37 AM","7/27/2021, 9:01:53 AM","7/27/2021, 9:02:29 AM","7/27/2021, 9:02:42 AM","7/27/2021, 9:02:50 AM","7/27/2021, 9:03:18 AM","7/27/2021, 9:03:22 AM","7/27/2021, 9:03:38 AM","7/27/2021, 9:03:50 AM","7/27/2021, 9:03:51 AM","7/27/2021, 9:03:58 AM","7/27/2021, 9:04:00 AM","7/27/2021, 9:04:01 AM","7/27/2021, 9:04:02 AM","7/27/2021, 9:04:06 AM","7/27/2021, 9:04:12 AM","7/27/2021, 9:04:24 AM","7/27/2021, 9:04:26 AM","7/27/2021, 9:04:27 AM","7/27/2021, 9:04:33 AM","7/27/2021, 9:04:59 AM","7/27/2021, 9:05:02 AM","7/27/2021, 9:05:05 AM","7/27/2021, 9:05:16 AM","7/27/2021, 9:05:20 AM","7/27/2021, 9:05:25 AM","7/27/2021, 9:05:31 AM","7/27/2021, 9:05:38 AM","7/27/2021, 9:05:44 AM","7/27/2021, 9:05:53 AM","7/27/2021, 9:06:03 AM","7/27/2021, 9:06:20 AM","7/27/2021, 9:06:21 AM","7/27/2021, 9:06:26 AM","7/27/2021, 9:06:30 AM","7/27/2021, 9:06:32 AM","7/27/2021, 9:06:39 AM","7/27/2021, 9:06:44 AM","7/27/2021, 9:06:47 AM","7/27/2021, 9:06:55 AM","7/27/2021, 9:07:04 AM","7/27/2021, 9:07:09 AM","7/27/2021, 9:07:14 AM","7/27/2021, 9:07:27 AM","7/27/2021, 9:07:29 AM","7/27/2021, 9:07:38 AM","7/27/2021, 9:07:43 AM","7/27/2021, 9:07:45 AM","7/27/2021, 9:07:46 AM","7/27/2021, 9:08:01 AM","7/27/2021, 9:08:08 AM","7/27/2021, 9:09:08 AM","7/27/2021, 9:09:20 AM","7/27/2021, 9:09:57 AM","7/27/2021, 9:10:51 AM","7/27/2021, 9:11:21 AM","7/27/2021, 9:13:37 AM","7/27/2021, 9:16:26 AM","7/27/2021, 9:17:34 AM","7/27/2021, 9:17:45 AM","7/27/2021, 9:18:00 AM","7/27/2021, 9:18:36 AM","7/27/2021, 9:20:15 AM","7/27/2021, 9:20:20 AM"
leave_time,"7/27/202

attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1).csv


,HUDIYA YASH,FERNANDO EDWIN,HANDA RHEA,gargi raje,CHARIVUKALAYIL JITIN,BANSAL SAHIL,KADU NITISH,PANDITA SHIVALIKA,HANAH SUSAN,DESHMUKH PARTH,SINGH ASHISH,KURIAN SANMITH,RIYA MOL,GUPTA PRANAY,KARAKATTIL DILROSE,ALWIN JACOB,JENNIFER JAMES,JUSTIN MATHEW,PINTO ALDRIC,SHYANN JAMES,GEORGE SIBIN,DANY POLY,FERNANDES ARYAN,KOTARKAR ANUJ,DANGE AFRIN,SHERIN SHIBU,RAGHUVANSHI AYUSH,SINGH JASBIR,BHOLR ADITI,SAWANT VISHADH,JERIN JOHN,BATTINA BABYKUTTY,WAREKAR ZAID,RADLEY ROY,MERWIN SAMUEL,BHAGYASHREE LAKSHMINARAYANAN,GOEL ESHAN,joel.thomas,shawn.periera,BHAGDIKAR SOUMITRA,RAMESH ANGELA,rahul.khambe,PATIL GAURI,SERENA RAJU,PADAMADAN SHAWN,SNEHA MARY,MELEDATH GARY,MULLICK SUDIKSHA,simran kalaskar,RAINA HARSHUL,MERIN ANN,B SAI SREE,MEHTA SHIVAM,JOSEPH BENJAMIN,GUPTA ISHIKA,THAKUR YUKTA,NAIR AKHIL,CHRISTOPHER CARDOZA,KURUTHUKULANGARA JULIE,GOLE SIDDHANT,GUPTA CHIRANYA,DSOUZA ARNOLD,SANKANNAWAR PRIYANKA,B SHRIRAM ATHREYA,KAKUSTE ANSHUTA,vedesh.nadar,PULHANI RACHIT,GEOFERRY MICHAEL,SHARMA AKSHAT
name,HUDIYA YASH,FERNANDO EDWIN,HANDA RHEA,gargi raje,CHARIVUKALAYIL JITIN,BANSAL SAHIL,KADU NITISH,PANDITA SHIVALIKA,HANAH SUSAN,DESHMUKH PARTH,SINGH ASHISH,KURIAN SANMITH,RIYA MOL,GUPTA PRANAY,KARAKATTIL DILROSE,ALWIN JACOB,JENNIFER JAMES,JUSTIN MATHEW,PINTO ALDRIC,SHYANN JAMES,GEORGE SIBIN,DANY POLY,FERNANDES ARYAN,KOTARKAR ANUJ,DANGE AFRIN,SHERIN SHIBU,RAGHUVANSHI AYUSH,SINGH JASBIR,BHOLR ADITI,SAWANT VISHADH,JERIN JOHN,BATTINA BABYKUTTY,WAREKAR ZAID,RADLEY ROY,MERWIN SAMUEL,BHAGYASHREE LAKSHMINARAYANAN,GOEL ESHAN,joel.thomas,shawn.periera,BHAGDIKAR SOUMITRA,RAMESH ANGELA,rahul.khambe,PATIL GAURI,SERENA RAJU,PADAMADAN SHAWN,SNEHA MARY,MELEDATH GARY,MULLICK SUDIKSHA,simran kalaskar,RAINA HARSHUL,MERIN ANN,B SAI SREE,MEHTA SHIVAM,JOSEPH BENJAMIN,GUPTA ISHIKA,THAKUR YUKTA,NAIR AKHIL,CHRISTOPHER CARDOZA,KURUTHUKULANGARA JULIE,GOLE SIDDHANT,GUPTA CHIRANYA,DSOUZA ARNOLD,SANKANNAWAR PRIYANKA,B SHRIRAM ATHREYA,KAKUSTE ANSHUTA,vedesh.nadar,PULHANI RACHIT,GEOFERRY MICHAEL,SHARMA AKSHAT
roll_no,101827,101817,101826,101870,101810,101804,101832,101845,101825,101814,101860,101836,101853,101824,101834,101801,101829,101831,101847,101859,101819,101813,101816,101835,101812,101858,101850,101861,101809,101855,101830,101805,101865,101849,101841,101808,101820,101775,101869,101807,101852,101867,101846,101856,101844,101862,101839,101842,101871,101851,101840,101802,101838,101806,101823,101863,101843,101710,101837,101821,101822,101815,101854,101803,101833,101868,101848,101818,101857
join_time,"7/28/2021, 1:41:35 PM","7/28/2021, 1:41:36 PM","7/28/2021, 1:41:45 PM","7/28/2021, 1:41:51 PM","7/28/2021, 1:42:14 PM","7/28/2021, 1:42:28 PM","7/28/2021, 1:42:29 PM","7/28/2021, 1:42:29 PM","7/28/2021, 1:43:07 PM","7/28/2021, 1:43:32 PM","7/28/2021, 1:43:34 PM","7/28/2021, 1:43:39 PM","7/28/2021, 1:43:46 PM","7/28/2021, 1:43:53 PM","7/28/2021, 1:43:57 PM","7/28/2021, 1:44:00 PM","7/28/2021, 1:44:07 PM","7/28/2021, 1:44:14 PM","7/28/2021, 1:44:15 PM","7/28/2021, 1:44:18 PM","7/28/2021, 1:44:20 PM","7/28/2021, 1:44:26 PM","7/28/2021, 1:44:29 PM","7/28/2021, 1:44:32 PM","7/28/2021, 1:44:35 PM","7/28/2021, 1:44:54 PM","7/28/2021, 1:44:54 PM","7/28/2021, 1:44:57 PM","7/28/2021, 1:44:59 PM","7/28/2021, 1:45:00 PM","7/28/2021, 1:45:02 PM","7/28/2021, 1:45:03 PM","7/28/2021, 1:45:04 PM","7/28/2021, 1:45:04 PM","7/28/2021, 1:45:04 PM","7/28/2021, 1:45:04 PM","7/28/2021, 1:45:05 PM","7/28/2021, 1:45:06 PM","7/28/2021, 1:45:24 PM","7/28/2021, 1:45:35 PM","7/28/2021, 1:45:35 PM","7/28/2021, 1:45:41 PM","7/28/2021, 1:45:43 PM","7/28/2021, 1:45:45 PM","7/28/2021, 1:45:50 PM","7/28/2021, 1:45:58 PM","7/28/2021, 1:45:59 PM","7/28/2021, 1:46:03 PM","7/28/2021, 1:46:06 PM","7/28/2021, 1:46:08 PM","7/28/2021, 1:46:14 PM","7/28/2021, 1:46:19 PM","7/28/2021, 1:46:22 PM","7/28/2021, 1:46:27 PM","7/28/2021, 1:46:27 PM","7/28/2021, 1:46:42 PM","7/28/2021, 1:46:47 PM","7/28/2021, 1:46:47 PM","7/28/2021, 1:47:34 PM","7/28/2021, 1:48:18 PM","7/28/2021, 1:48:20 PM","7/28/2021, 1:48:37 PM","7/28/2021, 1:48:56 PM","

attendance_analysis/data/BDA/meetingAttendanceReport(General) (1).csv


,B SAI SREE,simran kalaskar,MULLICK SUDIKSHA,FERNANDO EDWIN,SERENA RAJU,JUSTIN MATHEW,THAKUR YUKTA,CHRISTOPHER CARDOZA,HANAH SUSAN,JERIN JOHN,DANY POLY,DANGE AFRIN,GEOFERRY MICHAEL,gargi raje,JOSEPH BENJAMIN,BANSAL SAHIL,PINTO ALDRIC,KURIAN SANMITH,SAWANT VISHADH,JASON FELIX,SHERIN SHIBU,CHARIVUKALAYIL JITIN,GUPTA ISHIKA,RAMESH ANGELA,SHYANN JAMES,B SHRIRAM ATHREYA,GUPTA CHIRANYA,KAKUSTE ANSHUTA,SINGH ASHISH,VERMA VARUN,DSOUZA ARNOLD,PANDITA SHIVALIKA,rahul.khambe,BHOLR ADITI,FERNANDES ARYAN,PATIL GAURI,KADU NITISH,WAREKAR ZAID,GOEL ESHAN,RAGHUVANSHI AYUSH,SINGH JASBIR,GOLE SIDDHANT,MELEDATH GARY,MERIN ANN,BATTINA BABYKUTTY,GEORGE SIBIN,PULHANI RACHIT,BHAGDIKAR SOUMITRA,NAIR AKHIL,RAINA HARSHUL,SNEHA MARY,JENNIFER JAMES,KARAKATTIL DILROSE,SHARMA AKSHAT,BHAGYASHREE LAKSHMINARAYANAN,RADLEY ROY,DESHMUKH PARTH,SANKANNAWAR PRIYANKA,PADAMADAN SHAWN,joel.thomas,HUDIYA YASH,MEHTA SHIVAM,GUPTA PRANAY,KURUTHUKULANGARA JULIE,MERWIN SAMUEL,CHAVAN SHREYA
name,B SAI SREE,simran kalaskar,MULLICK SUDIKSHA,FERNANDO EDWIN,SERENA RAJU,JUSTIN MATHEW,THAKUR YUKTA,CHRISTOPHER CARDOZA,HANAH SUSAN,JERIN JOHN,DANY POLY,DANGE AFRIN,GEOFERRY MICHAEL,gargi raje,JOSEPH BENJAMIN,BANSAL SAHIL,PINTO ALDRIC,KURIAN SANMITH,SAWANT VISHADH,JASON FELIX,SHERIN SHIBU,CHARIVUKALAYIL JITIN,GUPTA ISHIKA,RAMESH ANGELA,SHYANN JAMES,B SHRIRAM ATHREYA,GUPTA CHIRANYA,KAKUSTE ANSHUTA,SINGH ASHISH,VERMA VARUN,DSOUZA ARNOLD,PANDITA SHIVALIKA,rahul.khambe,BHOLR ADITI,FERNANDES ARYAN,PATIL GAURI,KADU NITISH,WAREKAR ZAID,GOEL ESHAN,RAGHUVANSHI AYUSH,SINGH JASBIR,GOLE SIDDHANT,MELEDATH GARY,MERIN ANN,BATTINA BABYKUTTY,GEORGE SIBIN,PULHANI RACHIT,BHAGDIKAR SOUMITRA,NAIR AKHIL,RAINA HARSHUL,SNEHA MARY,JENNIFER JAMES,KARAKATTIL DILROSE,SHARMA AKSHAT,BHAGYASHREE LAKSHMINARAYANAN,RADLEY ROY,DESHMUKH PARTH,SANKANNAWAR PRIYANKA,PADAMADAN SHAWN,joel.thomas,HUDIYA YASH,MEHTA SHIVAM,GUPTA PRANAY,KURUTHUKULANGARA JULIE,MERWIN SAMUEL,CHAVAN SHREYA
roll_no,101802,101871,101842,101817,101856,101831,101863,101710,101825,101830,101813,101812,101818,101870,101806,101804,101847,101836,101855,101828,101858,101810,101823,101852,101859,101803,101822,101833,101860,101864,101815,101845,101867,101809,101816,101846,101832,101865,101820,101850,101861,101821,101839,101840,101805,101819,101848,101807,101843,101851,101862,101829,101834,101857,101808,101849,101814,101854,101844,101775,101827,101838,101824,101837,101841,101811
join_time,"7/27/2021, 9:00:54 AM","7/27/2021, 9:01:33 AM","7/27/2021, 9:01:37 AM","7/27/2021, 9:01:53 AM","7/27/2021, 9:02:29 AM","7/27/2021, 9:02:42 AM","7/27/2021, 9:02:50 AM","7/27/2021, 9:03:18 AM","7/27/2021, 9:03:22 AM","7/27/2021, 9:03:38 AM","7/27/2021, 9:03:50 AM","7/27/2021, 9:03:51 AM","7/27/2021, 9:03:58 AM","7/27/2021, 9:04:00 AM","7/27/2021, 9:04:01 AM","7/27/2021, 9:04:02 AM","7/27/2021, 9:04:06 AM","7/27/2021, 9:04:12 AM","7/27/2021, 9:04:24 AM","7/27/2021, 9:04:26 AM","7/27/2021, 9:04:27 AM","7/27/2021, 9:04:33 AM","7/27/2021, 9:04:59 AM","7/27/2021, 9:05:02 AM","7/27/2021, 9:05:05 AM","7/27/2021, 9:05:16 AM","7/27/2021, 9:05:20 AM","7/27/2021, 9:05:25 AM","7/27/2021, 9:05:31 AM","7/27/2021, 9:05:38 AM","7/27/2021, 9:05:44 AM","7/27/2021, 9:05:53 AM","7/27/2021, 9:06:03 AM","7/27/2021, 9:06:20 AM","7/27/2021, 9:06:21 AM","7/27/2021, 9:06:26 AM","7/27/2021, 9:06:30 AM","7/27/2021, 9:06:32 AM","7/27/2021, 9:06:39 AM","7/27/2021, 9:06:44 AM","7/27/2021, 9:06:47 AM","7/27/2021, 9:06:55 AM","7/27/2021, 9:07:04 AM","7/27/2021, 9:07:09 AM","7/27/2021, 9:07:14 AM","7/27/2021, 9:07:27 AM","7/27/2021, 9:07:29 AM","7/27/2021, 9:07:38 AM","7/27/2021, 9:07:43 AM","7/27/2021, 9:07:45 AM","7/27/2021, 9:07:46 AM","7/27/2021, 9:08:01 AM","7/27/2021, 9:08:08 AM","7/27/2021, 9:09:08 AM","7/27/2021, 9:09:20 AM","7/27/2021, 9:09:57 AM","7/27/2021, 9:10:51 AM","7/27/2021, 9:11:21 AM","7/27/2021, 9:13:37 AM","7/27/2021, 9:16:26 AM","7/27/2021, 9:17:34 AM","7/27/2021, 9:17:45 AM","7/27/2021, 9:18:00 AM","7/27/2021, 9:18:36 AM","7/27/2021, 9:20:15 AM","7/27/2021, 9:20:20 AM"
leave_time,"7/27/202

In [168]:
preprocessed_dirPath = 'attendance_analysis/preprocessed_data_json/'

# Preprocessing Data

In [189]:
def preprocessor(subject):
    dirPath = "attendance_analysis/data/"
    all_data = []
    for file in os.listdir(dirPath+subject):
        lecture_data = {}
        lecture_data['subject'] = subject
        if ".csv" in file:
            with open(dirPath+subject+"/"+file,'rt', encoding='utf16')as f:
                data = csv.reader(f, delimiter="\t")
                i = 0
                for row in data: # Add Metadata
                    if len(row) == 0:
                        skipped = i
                        break
                    if row[0] == 'Total Number of Participants':
                        lecture_data['participants'] = int(row[1])
                    if row[0] == 'Meeting Title':
                        lecture_data['meeting_title'] = row[1]
                    if row[0] == 'Meeting Start Time':
                        lecture_data['meeting_start_time'] = row[1]
                    if row[0] == 'Meeting End Time':
                        lecture_data['meeting_end_time'] = row[1]
                    i += 1
            print(dirPath+subject+"/"+file)      
            lecture = pd.read_csv(dirPath+subject+"/"+file, encoding = "UTF-16", skiprows=skipped, sep="\t")
        else:
            wb = xlrd.open_workbook(dirPath+subject+"/"+file)
            sheet = wb.sheet_by_index(0)
            skipped = 0
            for i in range(0,sheet.nrows): # Add Metadata
                row = sheet.row_values(i) 
                if row[0] == 'Total Number of Participants':
                    lecture_data['participants'] = int(row[1])
                if row[0] == 'Meeting Title':
                    lecture_data['meeting_title'] = row[1]
                if row[0] == 'Meeting Start Time':
                    lecture_data['meeting_start_time'] = row[1]
                if row[0] == 'Meeting End Time':
                    lecture_data['meeting_end_time'] = row[1]
                if row[0] == '':
                    skipped = i+1
                    break
            print(dirPath+subject+"/"+file)
            lecture = pd.read_excel(dirPath+subject+"/"+file, skiprows=skipped)
        attendees = {}
        for index,row in lecture.iterrows():
            if row['Full Name'] == 'pramod more':
                lecture_data['attendance_form_time'] = row['Join Time']
                break
            if row['Full Name'] not in attendees.keys():
                if row['Role'] == 'Presenter' or row['Role'] == 'Attendee':
                    if row['Full Name'] in roll_no.keys():
                        attendee = {}
                        attendee['name'] = row['Full Name']
                        attendee['roll_no'] = roll_no[row['Full Name']]
                        attendee['join_time'] = row['Join Time']
                        attendee['leave_time'] = row['Leave Time']
                        attendee['duration'] = time_parser(row['Duration'])
                        try:
                            attendee['email'] = row['Email']
                        except KeyError:
                            try: 
                                attendee['email'] = row['Email (userPrincipalName)']
                            except KeyError:
                                attendee['email'] = row['userPrincipalName']
                        if type(attendee['email']) is float:
                            attendee['email'] = '-'
                        attendees[row['Full Name']] = attendee
                else:
                    if 'organizer' not in lecture_data.keys():
                        organizer = {}
                        organizer['name'] = row['Full Name']
                        organizer['join_time'] = row['Join Time']
                        organizer['leave_time'] = row['Leave Time']
                        organizer['duration'] = time_parser(row['Duration'])
                        lecture_data['organizer'] = organizer
                    else:
                        organizer = lecture_data['organizer']
                        prev_join_time = datetime.strptime(organizer['join_time'], '%m/%d/%Y, %I:%M:%S %p')
                        curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
                        if prev_join_time < curr_join_time: 
                            organizer['leave_time'] = row['Leave Time']
                        else:
                            organizer['join_time'] = row['Join Time']
                        organizer['duration'] += time_parser(row['Duration'])
                        lecture_data['organizer'] = organizer
                        
            else: # multiple join-leave
                attendee = attendees[row['Full Name']]
                prev_join_time = datetime.strptime(attendee['join_time'], '%m/%d/%Y, %I:%M:%S %p')
                curr_join_time = datetime.strptime(row['Join Time'], '%m/%d/%Y, %I:%M:%S %p')
                if prev_join_time < curr_join_time: 
                    attendee['leave_time'] = row['Leave Time']
                else:
                    attendee['join_time'] = row['Join Time']
                attendee['duration'] += time_parser(row['Duration'])
        lecture_data['attendees'] = list(attendees.values())
        all_data.append(lecture_data)
    json_data = json.dumps(all_data, indent = 2)
    with open(preprocessed_dirPath+subject+"_data.json", "w") as outfile:
        outfile.write(json_data)
    print("\n\nFiled Created at -> ", preprocessed_dirPath+subject+"_data.json")

# Saving Data

In [190]:
bda_data = preprocessor("BDA")

attendance_analysis/data/BDA/meetingAttendanceReport(General) (2).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (2).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (5).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (3) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1) (2).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (4).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (2) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(General).csv
attendance_analysis/data/BDA/meetingAttendanceReport(General) (3).csv
attendance_analysis/data/BDA/meetingAttendanceReport(BDA(Theory)) (1).csv
attendance_analysis/data/BDA/meetingAttendanceReport(General) (1).csv


Filed Created at ->  attendance_analysis

In [191]:
dsip_data = preprocessor("DSIP")

attendance_analysis/data/DSIP/4-8-2021.csv
attendance_analysis/data/DSIP/25-8-2021 L no. 25.csv
attendance_analysis/data/DSIP/31-8-2021 Lno 28.csv
attendance_analysis/data/DSIP/17-8-2021-L no 22,23.csv
attendance_analysis/data/DSIP/5-8-2021.csv
attendance_analysis/data/DSIP/11-8-2021.csv
attendance_analysis/data/DSIP/16-8-2021-L no 21.csv
attendance_analysis/data/DSIP/24-8-2021- Lno 24.csv
attendance_analysis/data/DSIP/3-8-2021.csv
attendance_analysis/data/DSIP/26-8-2021 L no 26.csv
attendance_analysis/data/DSIP/6-8-2021.csv
attendance_analysis/data/DSIP/10-8-2021.csv


Filed Created at ->  attendance_analysis/preprocessed_data_json/DSIP_data.json


In [192]:
mcc_data = preprocessor("MCC")

attendance_analysis/data/MCC/MCC L23 26-08-2021.xlsx
attendance_analysis/data/MCC/MCC L31 16-09-2021.xlsx
attendance_analysis/data/MCC/MCC L12 29-07-2021.xlsx
attendance_analysis/data/MCC/MCC L2.xlsx
attendance_analysis/data/MCC/MCC L7.xlsx
attendance_analysis/data/MCC/MCC L28 08-09-2021.xlsx
attendance_analysis/data/MCC/MCC L33 17-09-2021.xlsx
attendance_analysis/data/MCC/MCC L22 25-08-2021.xlsx
attendance_analysis/data/MCC/MCC L14 04-08-2021.xlsx
attendance_analysis/data/MCC/MCC L24 27-08-2021.xlsx
attendance_analysis/data/MCC/MCC L9.xlsx
attendance_analysis/data/MCC/MCC L21 13-08-2021.xlsx
attendance_analysis/data/MCC/MCC L35 22-09-2021.xlsx
attendance_analysis/data/MCC/MCC L16 06-08-2021.xlsx
attendance_analysis/data/MCC/MCC L27 06-09-2021.xlsx
attendance_analysis/data/MCC/MCC L5.xlsx
attendance_analysis/data/MCC/MCC L18 10-08-2021.xlsx
attendance_analysis/data/MCC/MCC L17 09-08-2021.xlsx
attendance_analysis/data/MCC/MCC L10.xlsx
attendance_analysis/data/MCC/MCC L6.xlsx
attendance_